# Script to reclassify prints into prints, posters, and maybe artists books

Import libraries and define functions

In [ ]:
import pandas as pd

def csv_read(path: str, **kwargs) -> pd.DataFrame:
    """Loads a CSV table into a Pandas DataFrame with all cells as strings and blank cells as empty strings
    
    Keyword argument:
    rows -- the number of rows of the table to return when used for testing. When omitted, all rows are returned.
    """
    dataframe = pd.read_csv(path, na_filter=False, dtype = str)
    if 'rows' in kwargs:
        return dataframe.head(kwargs['rows']).copy(deep=True)
    else:
        return dataframe


Load data

In [ ]:
# Load the authoritative classification data from text_works_labels.csv file into a Pandas DataFrame
prints_metadata = csv_read('text_works_labels.csv')
prints_metadata = prints_metadata.set_index('qid')

# Load the existing metadata
path_to_vanderbot_metadata = '../gallery_works/works_multiprop.csv'
vanderbot_metadata = csv_read(path_to_vanderbot_metadata)
vanderbot_metadata = vanderbot_metadata.set_index('qid')




Extract the instance_of_uuid identifiers for the prints that need to be reclassified so that they can be deleted.

In [ ]:
# Create an empty dataframe to hold the qid and instance_of_uuid values for statements to be deleted
statements_to_delete = pd.DataFrame(columns=['qid', 'instance_of_uuid'])

# Loop through each row of the prints_metadata dataframe and find the instance_of_uuid value in the vanderbot_metadata dataframe
# If the instance_of_uuid value is found, copy the value of the instance_of column to the instance_of column in the prints_metadata dataframe
for index, row in prints_metadata.iterrows():
    # Screen for prints that have "poster" in the type column
    if 'poster' in row['type']:
        if index in vanderbot_metadata.index:
            statements_to_delete = statements_to_delete.append({'qid': index, 'instance_of_uuid': vanderbot_metadata.at[index, 'instance_of_uuid']}, ignore_index=True)

        # Check that the instance_of column has the value "Q11060274" (print)
        if row['instance_of'] != 'Q11060274':
            print('ERROR: ' + index + ' has instance_of value ' + row['instance_of'] + ' instead of Q11060274')
        else:
            # Change the value from "print" to "poster" (Q429785)
            vanderbot_metadata.at[index, 'instance_of'] = 'Q429785'
            # Change the value of the instance_of_uuid column to the empty string
            vanderbot_metadata.at[index, 'instance_of_uuid'] = ''
            # Change the value of the instance_of_ref1_hash column to the empty string
            vanderbot_metadata.at[index, 'instance_of_ref1_hash'] = ''

# Write the statements_to_delete dataframe to a CSV file
statements_to_delete.to_csv('deletions.csv', index=False)

# Write the updated vanderbot_metadata dataframe to a CSV file
vanderbot_metadata.to_csv(path_to_vanderbot_metadata, index=True)

After running this step, use vanderdeletebot.py to delete the P31 print statements.
Then run vanderbot.py to create the new P31 poster statements.